In [1]:
import sys
sys.path.append('../scripts/')
from read_utils import read_file, create_folder, temp_record_query, temp_record_sdf

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10GiB')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/05 01:02:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/05 01:02:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/05 01:02:59 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


- dataset

In [4]:
trans_sdf = read_file(spark, 'changed_data', '../data/curated/')

|> Loading File...


|> Loading Finished!


-RECORD 0--------------------------------------------------------------------------
 user_id           | 8972                                                          
 merchant_abn      | 59658193201                                                   
 dollar_value      | 182.16129389048888                                            
 order_id          | 5a2112b5-6a59-4ceb-95a5-655fa5915c7a                          
 merchant_name     | Id Ante Institute                                             
 tags              | cable, satellite, and other pay television and radio services 
 take_rate         | 6.16                                                          
 type              | a                                                             
 postcode          | 5311                                                          
 consumer_id       | 1115127                                                       
 name              | Mr. Bradley Moore                                      

In [5]:
merchant_fraud_sdf = read_file(spark,'merchant_fraud_probability.csv', '../data/tables/', 'csv')

|> Loading File...
|> Loading Finished!
-RECORD 0-------------------------------
 merchant_abn      | 19492220327        
 order_datetime    | 2021-11-28         
 fraud_probability | 44.403658647495355 
only showing top 1 row



In [6]:
merchant_fraud_sdf.show(5)

+------------+--------------+------------------+
|merchant_abn|order_datetime| fraud_probability|
+------------+--------------+------------------+
| 19492220327|    2021-11-28|44.403658647495355|
| 31334588839|    2021-10-02| 42.75530083865367|
| 19492220327|    2021-12-22|38.867790051131095|
| 82999039227|    2021-12-19|  94.1347004808891|
| 90918180829|    2021-09-02| 43.32551731714902|
+------------+--------------+------------------+
only showing top 5 rows



In [7]:
trans_sdf.show(5)

+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+--------+-----------+-----------------+--------------------+-----+------+----------+-----------------+---------------+----------+------------+-----------------+-----------+---------+--------+---------------+--------------+
|user_id|merchant_abn|      dollar_value|            order_id|       merchant_name|                tags|take_rate|type|postcode|consumer_id|             name|             address|state|gender|POA_CODE21|       population|New cases / day|     month|total_retail|__index_level_0__|genderIndex|typeIndex|tagIndex|        tag_vec|order_datetime|
+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+--------+-----------+-----------------+--------------------+-----+------+----------+-----------------+---------------+----------+------------+-----------------+-----------+---------+--------+------

In [8]:
sdf = trans_sdf.select( 'user_id', 'merchant_abn', 'order_datetime', 'dollar_value', 'take_rate', 'consumer_id', 'POA_CODE21', 'genderIndex', 'typeIndex', 'tagIndex', 'tag_vec')

In [9]:
sdf.show(5)

+-------+------------+--------------+------------------+---------+-----------+----------+-----------+---------+--------+---------------+
|user_id|merchant_abn|order_datetime|      dollar_value|take_rate|consumer_id|POA_CODE21|genderIndex|typeIndex|tagIndex|        tag_vec|
+-------+------------+--------------+------------------+---------+-----------+----------+-----------+---------+--------+---------------+
|   8972| 59658193201|    2021-11-26|182.16129389048888|     6.16|    1115127|      5311|        0.0|      0.0|     4.0| (24,[4],[1.0])|
|   9044| 83620670827|    2021-11-26| 93.18042034505228|     3.43|     916302|      3522|        1.0|      1.0|     4.0| (24,[4],[1.0])|
|   8968| 46887242827|    2021-11-26| 2132.463971960699|     2.43|     157590|      3483|        0.0|      2.0|    21.0|(24,[21],[1.0])|
|   8972| 86578477987|    2021-11-26| 75.91366457857609|     6.43|    1115127|      5311|        0.0|      0.0|     3.0| (24,[3],[1.0])|
|   8976| 15299889494|    2021-11-26|15.3

In [10]:
import pyspark.sql.functions as F
sdf_daily = sdf.groupby('merchant_abn', 'order_datetime', 'take_rate', 'tagIndex', 'tag_vec').agg(
    F.count("*").alias("count"), 
    F.avg("dollar_value").alias("dollar_average"),
    F.min("dollar_value").alias("dollar_min"),
    F.max("dollar_value").alias("dollar_max"),
    )

In [11]:
sdf_daily

merchant_abn,order_datetime,take_rate,tagIndex,tag_vec,count,dollar_average,dollar_min,dollar_max
25228131129,2021-11-26,4.5,22.0,"(24,[22],[1.0])",24,60.01129359973611,7.654280480650535,121.50995088141208
92343252737,2021-11-26,6.77,2.0,"(24,[2],[1.0])",48,83.84595082534592,36.3207433148876,137.94480883645028
74589998324,2021-11-26,6.8,20.0,"(24,[20],[1.0])",11,245.17230409345208,32.48881618941423,482.11873205344796
49069349536,2021-11-26,0.61,22.0,"(24,[22],[1.0])",4,3586.5844876420038,383.3917332398315,5457.035558696636
23026499747,2021-11-26,6.95,3.0,"(24,[3],[1.0])",18,37.46845702633031,8.447219506788867,133.58116220242246
45648966421,2021-11-26,5.85,1.0,"(24,[1],[1.0])",6,71.35287784278087,41.96184941443322,127.70068814481043
20036872831,2021-11-26,2.75,2.0,"(24,[2],[1.0])",3,81.82014577128139,60.716739823514466,123.73231493301282
73918814753,2021-11-26,4.99,10.0,"(24,[10],[1.0])",2,2166.480650893464,2020.7800023776051,2312.181299409323
15795298692,2021-11-26,3.29,23.0,"(24,[23],[1.0])",1,3436.7801997952156,3436.7801997952156,3436.7801997952156
77071952077,2021-11-26,5.91,4.0,"(24,[4],[1.0])",1,157.0,157.0,157.0


In [12]:
merchant_fraud_sdf.show

<bound method DataFrame.show of +------------+--------------+------------------+
|merchant_abn|order_datetime| fraud_probability|
+------------+--------------+------------------+
| 19492220327|    2021-11-28|44.403658647495355|
| 31334588839|    2021-10-02| 42.75530083865367|
| 19492220327|    2021-12-22|38.867790051131095|
| 82999039227|    2021-12-19|  94.1347004808891|
| 90918180829|    2021-09-02| 43.32551731714902|
| 31334588839|    2021-12-26| 38.36165958070444|
| 23686790459|    2021-12-10|  79.4543441508535|
| 14827550074|    2021-11-26| 46.45775596795885|
| 31334588839|    2021-11-26| 36.20971272078342|
| 19492220327|    2021-12-18|33.819672154331755|
| 31334588839|    2021-11-29|35.386213297375505|
| 14827550074|    2021-12-05| 43.85519494291279|
| 19492220327|    2021-11-18|32.193139919494016|
| 93260930990|    2021-11-30| 37.87197154172081|
| 90918180829|    2021-09-16| 36.62001350882694|
| 83199298021|    2022-02-27|26.025158824861773|
| 83199298021|    2022-02-17| 25.7799

In [13]:
train_data = sdf_daily.join(merchant_fraud_sdf, on=['merchant_abn', 'order_datetime'], how = 'inner')

In [14]:
trans_data = sdf_daily.join(merchant_fraud_sdf, on=['merchant_abn', 'order_datetime'], how = 'left_outer')

In [15]:
train_data

merchant_abn,order_datetime,take_rate,tagIndex,tag_vec,count,dollar_average,dollar_min,dollar_max,fraud_probability
93292821052,2021-11-29,1.54,10.0,"(24,[10],[1.0])",8,412.20693383334697,189.92064574813463,587.1473799860383,66.58725735032715
90568944804,2021-11-29,4.1,0.0,"(24,[0],[1.0])",49,1053.7868134661658,44.35499568574597,4691.359952452361,33.172494688960434
31334588839,2021-12-18,4.22,19.0,"(24,[19],[1.0])",6,7494.506190165383,251.86867522240155,19238.0352453035,27.53494190354933
15043504837,2021-12-14,4.62,24.0,"(24,[],[])",2,23282.021588214167,20937.32335534359,25626.719821084745,26.12523097610844
11590404675,2021-12-21,4.19,19.0,"(24,[19],[1.0])",2,25903.860931071147,21317.87317143576,30489.848690706538,29.607818240092094
19492220327,2021-12-22,4.93,24.0,"(24,[],[])",4,19385.860069909955,11023.51582242438,38156.49714654181,38.867790051131095
14530561097,2021-09-15,1.69,24.0,"(24,[],[])",1,46001.13901942742,46001.13901942742,46001.13901942742,80.80054474543395
83199298021,2022-02-17,4.82,19.0,"(24,[19],[1.0])",2,29419.854525584382,25769.16321318206,33070.545837986705,25.77998392496447
31334588839,2021-11-26,4.22,19.0,"(24,[19],[1.0])",6,10932.610545316842,150.03930077561475,36517.48082996128,36.20971272078342
72058040180,2021-12-11,6.77,19.0,"(24,[19],[1.0])",2,27184.041478159186,23543.554827302272,30824.5281290161,33.71875978884334


In [16]:
trans_data

merchant_abn,order_datetime,take_rate,tagIndex,tag_vec,count,dollar_average,dollar_min,dollar_max,fraud_probability
25228131129,2021-11-26,4.5,22.0,"(24,[22],[1.0])",24,60.01129359973611,7.654280480650535,121.50995088141208,null
92343252737,2021-11-26,6.77,2.0,"(24,[2],[1.0])",48,83.84595082534592,36.3207433148876,137.94480883645028,null
74589998324,2021-11-26,6.8,20.0,"(24,[20],[1.0])",11,245.17230409345208,32.48881618941423,482.11873205344796,null
49069349536,2021-11-26,0.61,22.0,"(24,[22],[1.0])",4,3586.5844876420038,383.3917332398315,5457.035558696636,null
23026499747,2021-11-26,6.95,3.0,"(24,[3],[1.0])",18,37.46845702633031,8.447219506788867,133.58116220242246,null
45648966421,2021-11-26,5.85,1.0,"(24,[1],[1.0])",6,71.35287784278087,41.96184941443322,127.70068814481043,null
20036872831,2021-11-26,2.75,2.0,"(24,[2],[1.0])",3,81.82014577128139,60.716739823514466,123.73231493301282,null
73918814753,2021-11-26,4.99,10.0,"(24,[10],[1.0])",2,2166.480650893464,2020.7800023776051,2312.181299409323,null
15795298692,2021-11-26,3.29,23.0,"(24,[23],[1.0])",1,3436.7801997952156,3436.7801997952156,3436.7801997952156,null
77071952077,2021-11-26,5.91,4.0,"(24,[4],[1.0])",1,157.0,157.0,157.0,null


In [17]:
from pyspark.sql.functions import year, month, dayofmonth

In [18]:
train_fraud_table = (train_data
               .withColumn('order_year', year(train_data.order_datetime))
               .withColumn('order_month', month(train_data.order_datetime))
               .withColumn('order_day', dayofmonth(train_data.order_datetime))
               )

In [19]:
train_fraud_table = train_fraud_table.drop('order_datetime')

In [20]:
train_fraud_table.show(5)

+------------+---------+--------+---------------+-----+------------------+------------------+------------------+------------------+----------+-----------+---------+
|merchant_abn|take_rate|tagIndex|        tag_vec|count|    dollar_average|        dollar_min|        dollar_max| fraud_probability|order_year|order_month|order_day|
+------------+---------+--------+---------------+-----+------------------+------------------+------------------+------------------+----------+-----------+---------+
| 93292821052|     1.54|    10.0|(24,[10],[1.0])|    8|412.20693383334697|189.92064574813463| 587.1473799860383| 66.58725735032715|      2021|         11|       29|
| 90568944804|      4.1|     0.0| (24,[0],[1.0])|   49|1053.7868134661658| 44.35499568574597| 4691.359952452361|33.172494688960434|      2021|         11|       29|
| 31334588839|     4.22|    19.0|(24,[19],[1.0])|    6| 7494.506190165383|251.86867522240155|  19238.0352453035| 27.53494190354933|      2021|         12|       18|
| 15043504

In [21]:
train_fraud_table = (train_fraud_table
                     .withColumn('fraud_probability', train_fraud_table['fraud_probability'].cast('double')))

- Test Data

In [22]:
test_fraud_table = (trans_data
               .withColumn('order_year', year(train_data.order_datetime))
               .withColumn('order_month', month(train_data.order_datetime))
               .withColumn('order_day', dayofmonth(train_data.order_datetime))
               )

In [23]:
test_fraud_table = test_fraud_table.drop('order_datetime','fraud_probability')

In [24]:
test_fraud_table.show(5)

+------------+---------+--------+---------------+-----+------------------+-----------------+------------------+----------+-----------+---------+
|merchant_abn|take_rate|tagIndex|        tag_vec|count|    dollar_average|       dollar_min|        dollar_max|order_year|order_month|order_day|
+------------+---------+--------+---------------+-----+------------------+-----------------+------------------+----------+-----------+---------+
| 25228131129|      4.5|    22.0|(24,[22],[1.0])|   24| 60.01129359973611|7.654280480650535|121.50995088141208|      2021|         11|       26|
| 92343252737|     6.77|     2.0| (24,[2],[1.0])|   48| 83.84595082534592| 36.3207433148876|137.94480883645028|      2021|         11|       26|
| 74589998324|      6.8|    20.0|(24,[20],[1.0])|   11|245.17230409345208|32.48881618941423|482.11873205344796|      2021|         11|       26|
| 49069349536|     0.61|    22.0|(24,[22],[1.0])|    4|3586.5844876420038|383.3917332398315| 5457.035558696636|      2021|        

- Model

In [25]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [26]:
input_col = ['merchant_abn', 'take_rate', 'tagIndex', 'tag_vec', 'count', 'dollar_average', 'dollar_min', 'dollar_max', 'order_year', 'order_month', 'order_day']

In [27]:
vecAssembler = VectorAssembler(inputCols=input_col, outputCol="features")

In [28]:
train_dataset = vecAssembler.transform(train_fraud_table)

In [29]:
train_dataset.show(5)

+------------+---------+--------+---------------+-----+------------------+------------------+------------------+------------------+----------+-----------+---------+--------------------+
|merchant_abn|take_rate|tagIndex|        tag_vec|count|    dollar_average|        dollar_min|        dollar_max| fraud_probability|order_year|order_month|order_day|            features|
+------------+---------+--------+---------------+-----+------------------+------------------+------------------+------------------+----------+-----------+---------+--------------------+
| 93292821052|     1.54|    10.0|(24,[10],[1.0])|    8|412.20693383334697|189.92064574813463| 587.1473799860383| 66.58725735032715|      2021|         11|       29|(34,[0,1,2,13,27,...|
| 90568944804|      4.1|     0.0| (24,[0],[1.0])|   49|1053.7868134661658| 44.35499568574597| 4691.359952452361|33.172494688960434|      2021|         11|       29|(34,[0,1,3,27,28,...|
| 31334588839|     4.22|    19.0|(24,[19],[1.0])|    6| 7494.506190165

In [30]:
log_reg = LinearRegression(labelCol = 'fraud_probability').fit(train_dataset)

22/10/05 01:05:16 WARN Instrumentation: [10dd53ab] regParam is zero, which might cause numerical instability and overfitting.
22/10/05 01:05:16 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/10/05 01:05:16 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/10/05 01:05:16 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
22/10/05 01:05:16 WARN Instrumentation: [10dd53ab] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


22/10/05 01:05:17 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search zoom failed
22/10/05 01:05:17 ERROR LBFGS: Failure again! Giving up and returning. Maybe the objective is just poorly behaved?


In [31]:
test_feature = vecAssembler.transform(test_fraud_table)

In [32]:
test_feature.show(5)

+------------+---------+--------+---------------+-----+------------------+-----------------+------------------+----------+-----------+---------+--------------------+
|merchant_abn|take_rate|tagIndex|        tag_vec|count|    dollar_average|       dollar_min|        dollar_max|order_year|order_month|order_day|            features|
+------------+---------+--------+---------------+-----+------------------+-----------------+------------------+----------+-----------+---------+--------------------+
| 25228131129|      4.5|    22.0|(24,[22],[1.0])|   24| 60.01129359973611|7.654280480650535|121.50995088141208|      2021|         11|       26|(34,[0,1,2,25,27,...|
| 92343252737|     6.77|     2.0| (24,[2],[1.0])|   48| 83.84595082534592| 36.3207433148876|137.94480883645028|      2021|         11|       26|(34,[0,1,2,5,27,2...|
| 74589998324|      6.8|    20.0|(24,[20],[1.0])|   11|245.17230409345208|32.48881618941423|482.11873205344796|      2021|         11|       26|(34,[0,1,2,23,27,...|
| 49

In [33]:
test_pred = log_reg.transform(test_feature)

In [34]:
test_pred.show(5)

+------------+---------+--------+---------------+-----+------------------+-----------------+------------------+----------+-----------+---------+--------------------+------------------+
|merchant_abn|take_rate|tagIndex|        tag_vec|count|    dollar_average|       dollar_min|        dollar_max|order_year|order_month|order_day|            features|        prediction|
+------------+---------+--------+---------------+-----+------------------+-----------------+------------------+----------+-----------+---------+--------------------+------------------+
| 25228131129|      4.5|    22.0|(24,[22],[1.0])|   24| 60.01129359973611|7.654280480650535|121.50995088141208|      2021|         11|       26|(34,[0,1,2,25,27,...| 32.69619266265363|
| 92343252737|     6.77|     2.0| (24,[2],[1.0])|   48| 83.84595082534592| 36.3207433148876|137.94480883645028|      2021|         11|       26|(34,[0,1,2,5,27,2...|36.614809423772385|
| 74589998324|      6.8|    20.0|(24,[20],[1.0])|   11|245.17230409345208|3

In [35]:
test_pred = test_pred.withColumnRenamed('prediction', 'fraud_probability')

In [36]:
test_pred.show(5)

+------------+---------+--------+---------------+-----+------------------+-----------------+------------------+----------+-----------+---------+--------------------+------------------+
|merchant_abn|take_rate|tagIndex|        tag_vec|count|    dollar_average|       dollar_min|        dollar_max|order_year|order_month|order_day|            features| fraud_probability|
+------------+---------+--------+---------------+-----+------------------+-----------------+------------------+----------+-----------+---------+--------------------+------------------+
| 25228131129|      4.5|    22.0|(24,[22],[1.0])|   24| 60.01129359973611|7.654280480650535|121.50995088141208|      2021|         11|       26|(34,[0,1,2,25,27,...| 32.69619266265363|
| 92343252737|     6.77|     2.0| (24,[2],[1.0])|   48| 83.84595082534592| 36.3207433148876|137.94480883645028|      2021|         11|       26|(34,[0,1,2,5,27,2...|36.614809423772385|
| 74589998324|      6.8|    20.0|(24,[20],[1.0])|   11|245.17230409345208|3

In [42]:
test_pred.count()

389413

In [41]:
temp_record_sdf(test_pred, '../data/curated/merchant_fraud.parquet', True)

|> Waitting for saving...
|> The folder already exist! Change the attr `overwrite` to cover the origin data.
